In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [45]:
#pip install pandas openpyxl

In [21]:
District_23 = pd.read_excel('data/Education 2023_24/Accountablity/District_level/district_release_file_suppressed_2023.xlsx')
#school_24 = pd.read_excel('data/Education 2023_24/Accountablity/school_level/school_release_file_suppressed_2024.xlsx')


In [25]:
District_23.head(1
            )

,year,system,system_name,subject,grade,student_group,valid_tests,n_below,n_approaching,n_met_expectations,n_exceeded_expectations,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_met_exceeded
0,2023,10,Anderson County,Biology I,9th through 12th,All Students,438,94,137,176,31,21.4,31.3,40.2,7.1,47.3


In [75]:
#convert numeric columns to numeric and replece suppressed entries to NaN
District_23['valid_tests'] = pd.to_numeric(District_23['valid_tests'], errors='coerce')
District_23['n_below'] = pd.to_numeric(District_23['n_below'], errors='coerce')
District_23['n_approaching'] = pd.to_numeric(District_23['n_approaching'], errors='coerce')
District_23['n_met_expectations'] = pd.to_numeric(District_23['n_met_expectations'], errors='coerce')
District_23['n_exceeded_expectations'] = pd.to_numeric(District_23['n_exceeded_expectations'], errors='coerce')

In [77]:
#group and aggreage the df
District_total_23 = District_23.groupby(['year','system','system_name','student_group']).agg(
                    valid_tests = ('valid_tests', 'sum'),
                    n_below = ('n_below','sum'),
                    n_approaching = ('n_approaching','sum'),
                    n_met_expectations = ('n_met_expectations','sum'),
                    n_exceeded_expectations = ('n_exceeded_expectations','sum')).reset_index()
District_total_23.head()

,year,system,system_name,student_group,valid_tests,n_below,n_approaching,n_met_expectations,n_exceeded_expectations
0,2023,10,Anderson County,All Students,21775,5093.0,8512.0,6500.0,1670.0
1,2023,10,Anderson County,Black/Hispanic/Native American,1908,545.0,763.0,480.0,88.0
2,2023,10,Anderson County,Economically Disadvantaged,6540,1990.0,2733.0,1437.0,266.0
3,2023,10,Anderson County,English Learners with Transitional 1-4,437,78.0,104.0,38.0,8.0
4,2023,10,Anderson County,Students with Disabilities,3758,1734.0,1173.0,436.0,102.0


In [113]:
# Calculate passed students
District_total_23['passed'] = District_total_23['n_met_expectations'] + District_total_23['n_exceeded_expectations']

# Calculate percentage passed with numeric formatting
District_total_23['pct_passed'] = (District_total_23['passed'] * 100 / District_total_23['valid_tests']).round(2)

# Filter for 'All Students'
District_total_23_filtered = District_total_23.loc[District_total_23['student_group'] == 'All Students']

# Sort by 'pct_passed' descending
District_total_23_sorted = District_total_23_filtered.sort_values(by='pct_passed', ascending=False)

District_total_23_sorted.head(30)


,year,system,system_name,student_group,valid_tests,n_below,n_approaching,n_met_expectations,n_exceeded_expectations,passed,pct_passed
813,2023,940,Williamson County,All Students,152273,9328.0,33718.0,69697.0,39530.0,109227.0,71.73
48,2023,52,Maryville,All Students,20249,1709.0,4643.0,9014.0,4881.0,13895.0,68.62
694,2023,796,Germantown,All Students,20983,1560.0,5135.0,9444.0,4844.0,14288.0,68.09
688,2023,795,Collierville,All Students,33286,2681.0,8232.0,13562.0,8811.0,22373.0,67.21
700,2023,797,Lakeland,All Students,9643,716.0,2582.0,4141.0,2204.0,6345.0,65.80
676,2023,793,Arlington,All Students,16108,1371.0,4380.0,7170.0,3187.0,10357.0,64.30
789,2023,901,Johnson City,All Students,28020,3515.0,7316.0,10659.0,6530.0,17189.0,61.35
819,2023,941,Franklin SSD,All Students,13516,1622.0,3810.0,5420.0,2664.0,8084.0,59.81
275,2023,275,Gibson Co Sp Dist,All Students,14416,1761.0,4587.0,5895.0,2173.0,8068.0,55.97
825,2023,950,Wilson County,All Students,69650,9536.0,21456.0,27387.0,11271.0,38658.0,55.50


In [115]:
District_total_23_sorted[(District_total_23_sorted['system_name']=='Rutherford County') | (District_total_23_sorted['system_name']=='Davidson County')]

,year,system,system_name,student_group,valid_tests,n_below,n_approaching,n_met_expectations,n_exceeded_expectations,passed,pct_passed
634,2023,750,Rutherford County,All Students,187153,35154.0,63310.0,64999.0,23690.0,88689.0,47.39
197,2023,190,Davidson County,All Students,258310,92717.0,93963.0,55814.0,15816.0,71630.0,27.73


In [ ]:
# add zipcode column

In [ ]:
#add student attendance column

In [ ]:
#add student cronic absenteeism column

In [ ]:
#add crime rating column